<a href="https://colab.research.google.com/github/rh2098ta/practicalapplication2/blob/main/practicalassignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
aimport pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('vehicles.csv')
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'vehicles.csv'

In [ ]:
# shape, columns, nulls, data types
print(df.shape)
print(df.dtypes)
df.isnull().sum()


In [ ]:
import matplotlib.pyplot as plt

plt.hist(df['price'], bins=50)
plt.title('Distribution of Used Car Prices')
plt.xlabel('Price')
plt.ylabel('Count')
plt.show()


In [ ]:
df['price'] = pd.to_numeric(df['price'], errors='coerce')
print(df['price'].dtype)
print(df['price'].describe())
df['price'].nlargest(10)


In [ ]:
df_price_ok = df[(df['price'] >= 500) & (df['price'] <= 200_000)].copy()


In [ ]:
lo, hi = df['price'].quantile([0.01, 0.99])
df_price_ok = df[df['price'].between(lo, hi)].copy()
lo, hi


In [ ]:
plt.hist(df_price_ok['price'], bins=60)
plt.title('Distribution of Used Car Prices')
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.show()



In [ ]:
df_price_ok['odometer'] = pd.to_numeric(df_price_ok['odometer'], errors='coerce')
df_price_ok = df_price_ok[(df_price_ok['odometer'] >= 0) & (df_price_ok['odometer'] <= 400_000)]
df_price_ok = df_price_ok[(df_price_ok['year'] >= 1985) & (df_price_ok['year'] <= 2025)]


In [ ]:
df_clean = df_price_ok.copy()
df_clean.info()
df_clean.describe(include='all')



In [ ]:
features = ['year', 'odometer', 'manufacturer', 'model', 'condition', 'fuel',
            'transmission', 'type', 'paint_color', 'state']

df_model = df_clean[features + ['price']].dropna()
df_model.head()


In [ ]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("MAE:", mae)
print("RMSE:", rmse)

In [ ]:
ct = model.named_steps['preprocess']
feature_names = ct.get_feature_names_out()
coefs = model.named_steps['reg'].coef_.ravel()

feature_importance = (
    pd.DataFrame({'feature': feature_names, 'coefficient': coefs})
      .sort_values('coefficient', ascending=False)
)

print("Top 10 features increasing price:")
display(feature_importance.head(10))

print("Top 10 features decreasing price:")
display(feature_importance.tail(10))


In [ ]:
print("MAE (Mean Absolute Error):", mae)
print("RMSE (Root Mean Squared Error):", rmse)


ohe = model.named_steps['preprocess'].named_transformers_['cat']
encoded_cols = ohe.get_feature_names_out(categorical_features)
all_feature_names = list(encoded_cols) + numeric_features

coeffs = model.named_steps['reg'].coef_

feature_importance = (
    pd.DataFrame({'feature': all_feature_names, 'coefficient': coeffs})
    .sort_values('coefficient', ascending=False)
)

print("\nTop 10 features increasing price:")
display(feature_importance.head(10))

print("\nTop 10 features decreasing price:")
display(feature_importance.tail(10))


In [ ]:
try:
    df_viz = df_price_ok.copy()
except NameError:
    try:
        df_viz = df_clean.copy()
    except NameError:
        df_viz = df.copy()

for col in ['price','odometer','year']:
    if col in df_viz.columns:
        df_viz[col] = pd.to_numeric(df_viz[col], errors='coerce')

if 'price' in df_viz.columns:
    df_viz = df_viz[(df_viz['price'] >= 500) & (df_viz['price'] <= 200000)]
if 'odometer' in df_viz.columns:
    df_viz = df_viz[(df_viz['odometer'] >= 0) & (df_viz['odometer'] <= 400000)]
if 'year' in df_viz.columns:
    df_viz = df_viz[(df_viz['year'] >= 1985) & (df_viz['year'] <= 2025)]

df_viz = df_viz.dropna(subset=['price'])
df_viz.shape


In [ ]:
import seaborn as sns


if 'odometer' in df_viz.columns:
    plt.figure(figsize=(8,5))
    sns.scatterplot(data=df_viz.sample(min(5000, len(df_viz)), random_state=42),
                    x='odometer', y='price', alpha=0.3)
    plt.title('Price vs Odometer')
    plt.xlabel('Odometer (miles)')
    plt.ylabel('Price ($)')
    plt.tight_layout()
    plt.show()

if 'type' in df_viz.columns:
    top_types = df_viz['type'].value_counts().head(8).index
    plt.figure(figsize=(10,6))
    sns.boxplot(data=df_viz[df_viz['type'].isin(top_types)],
                x='type', y='price')
    plt.title('Price by Vehicle Type (Top 8 Types)')
    plt.xlabel('Vehicle Type')
    plt.ylabel('Price ($)')
    plt.xticks(rotation=20)
    plt.tight_layout()
    plt.show()

if 'fuel' in df_viz.columns:
    plt.figure(figsize=(8,5))
    sns.boxplot(data=df_viz, x='fuel', y='price')
    plt.title('Price by Fuel Type')
    plt.xlabel('Fuel')
    plt.ylabel('Price ($)')
    plt.tight_layout()
    plt.show()


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

df_model = df_viz.copy()
current_year = 2025
df_model['car_age'] = current_year - pd.to_numeric(df_model['year'], errors='coerce')

feature_candidates = [
    'car_age','odometer','manufacturer','model','condition','fuel',
    'transmission','type','paint_color','state'
]
features = [c for c in feature_candidates if c in df_model.columns]

top_n = 200
top_models = df_model['model'].value_counts().head(top_n).index
df_model['model_slim'] = np.where(df_model['model'].isin(top_models), df_model['model'], 'other')
features = [c for c in features if c != 'model'] + ['model_slim']

df_model = df_model.dropna(subset=features + ['price']).copy()

target_rows = 120000
if len(df_model) > target_rows:
    df_model = df_model.sample(n=target_rows, random_state=42)

X = df_model[features].copy()
y = pd.to_numeric(df_model['price'], errors='coerce')

numeric_features = [c for c in ['car_age','odometer'] if c in X.columns]
categorical_features = [c for c in X.columns if c not in numeric_features]

ohe = OneHotEncoder(handle_unknown='ignore')

preprocess = ColumnTransformer(
    transformers=[('cat', ohe, categorical_features)],
    remainder='passthrough'
)

lin_model = Pipeline(steps=[
    ('preprocess', preprocess),
    ('reg', LinearRegression())
])

rf_model = Pipeline(steps=[
    ('preprocess', preprocess),
    ('rf', RandomForestRegressor(
        n_estimators=150,
        max_depth=18,
        max_features='sqrt',
        min_samples_leaf=3,
        n_jobs=-1,
        random_state=42
    ))
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

lin_model.fit(X_train, y_train)
lin_pred = lin_model.predict(X_test)
lin_mae = mean_absolute_error(y_test, lin_pred)
lin_rmse = np.sqrt(mean_squared_error(y_test, lin_pred))
print("Linear Regression - MAE:", lin_mae, "RMSE:", lin_rmse)

rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_mae = mean_absolute_error(y_test, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
print("Random Forest      - MAE:", rf_mae, "RMSE:", rf_rmse)




In [ ]:
from sklearn.model_selection import cross_val_score

lin_cv = -cross_val_score(lin_model, X, y,
                          scoring='neg_mean_absolute_error',
                          cv=5, n_jobs=-1)
print("Linear CV MAE mean:", lin_cv.mean())


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'rf__n_estimators': [80, 120]
}

rf_gs = GridSearchCV(
    rf_model,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=2,
    n_jobs=-1,
    verbose=1
)

rf_gs.fit(X_train, y_train)
best_rf = rf_gs.best_estimator_

best_pred = best_rf.predict(X_test)
best_mae = mean_absolute_error(y_test, best_pred)
best_rmse = np.sqrt(mean_squared_error(y_test, best_pred))

print("Best params:", rf_gs.best_params_)
print("Best RF MAE:", best_mae, "Best RF RMSE:", best_rmse)


In [ ]:
plt.figure(figsize=(7,5))
sns.barplot(x='fuel', y='price', data=df_viz, estimator=np.mean, errorbar=None)
plt.title('Average Price by Fuel Type')
plt.xlabel('Fuel')
plt.ylabel('Average Price')
plt.tight_layout()
plt.show()
